In [1]:
import re
import requests
import json

import pymysql

# your username and password go here!
creds = {'user': 'brian', 
         'pswd': 'farley'} 


connection = pymysql.connect(host="atriskwiki.cbprs6vpqcbz.us-west-1.rds.amazonaws.com",
                            user=creds['user'],
                            password=creds['pswd'],
                            port=3306,
                            db="atriskwiki",
                            charset="utf8")

cur = connection.cursor()

In [2]:
monthpart = r"""(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)"""
daypart = r"\d{1,2}\w{0,2}"
yearpart = "\d{4}"

america = r"{} {},? {}".format(monthpart, daypart, yearpart)
europe = r"{} {} {}".format(daypart, monthpart, yearpart)

print(america)
print(europe)

amdate = re.compile(america, re.I)
eurodate = re.compile(europe, re.I)

(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?) \d{1,2}\w{0,2},? \d{4}
\d{1,2}\w{0,2} (?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?) \d{4}


In [3]:
epafulltextsURL = "https://storage.googleapis.com/atriskwiki/epa_fulltext.txt"
epafulltexts = requests.get(epafulltextsURL).text.split("\n")
nepafulltextsURL = "https://storage.googleapis.com/atriskwiki/nepa_fulltext.txt"
nepafulltexts = requests.get(nepafulltextsURL).text.split("\n")

In [25]:
extract_num = 666
trial = json.loads(fulltexts[extract_num])
text = trial[list(trial.keys())[0]]
print(text)

The Palestine Liberation Organization (PLO) (Arabic: منظمة التحرير الفلسطينية‎‎;  Munaẓẓamat at-Taḥrīr al-Filasṭīniyyah ) is an organization founded in 1964 with the purpose of the "liberation of Palestine" through armed struggle. It is recognized as the "sole legitimate representative of the Palestinian people" by over 100 states with which it holds diplomatic relations, and has enjoyed observer status at the United Nations since 1974. The PLO was considered by the United States and Israel to be a terrorist organization until the Madrid Conference in 1991. In 1993, the PLO recognized Israel's right to exist in peace, accepted UN Security Council resolutions 242 and 338, and rejected "violence and terrorism"; in response, Israel officially recognized the PLO as the representative of the Palestinian people.


== FoundingEdit ==
At its first summit meeting in Cairo in 1964, the Arab League initiated the creation of an organization representing the Palestinian people. The Palestinian Nati

In [26]:
eumatches = re.findall(eurodate, text)

In [27]:
eumatches

['28 May 1964',
 '2 June 1964',
 '4 February 1969',
 '22 November 1974',
 '12 January 1976',
 '2 April 1986',
 '7 July 1998',
 '29 November 2012',
 '20 July 2010',
 '9 June 1974',
 '20 August 1993',
 '13 September 1993',
 '20 January 1976',
 '1 October 1985',
 '2 June 1964',
 '24 December 1967',
 '24 December 1967',
 '2 February 1969',
 '4 February 1969',
 '11 November 2004',
 '29 October 2004',
 '11 November 2004',
 '8 June 1974',
 '8 June 2005']

In [28]:
ammatches = re.findall(amdate, text)

In [29]:
ammatches

['January 18, 2008']

In [66]:
def GetDatesFromArticle(articleText):
    '''Note: the in context search for dates DOES NOT search through newline characters!'''
    dates ={'amdates':re.findall(amdate, articleText),
            'eurodates':re.findall(eurodate, articleText)}
    for date in dates['amdates']:
        context = re.compile(r".{0,100}" + date + r".{0,100}")
        m = re.search(context, articleText)
        dates[date] = m.group(0)
    for date in dates['eurodates']:
        context = re.compile(r".{0,100}" + date + r".{0,100}")
        m = re.search(context, articleText)
        dates[date] = m.group(0) 
    return dates

In [67]:
foundDates = dict()
for article in nepafulltexts:
    try:
        json_article = json.loads(article)
        article_id = list(json_article.keys())[0]
        foundDates[article_id] = GetDatesFromArticle(json_article[article_id])
    except:
        pass

for article in epafulltexts:
    try:
        json_article = json.loads(article)
        article_id = list(json_article.keys())[0]
        foundDates[article_id] = GetDatesFromArticle(json_article[article_id])
    except:
        pass


In [38]:
test = foundDates[list(foundDates.keys())[50002]]
test

{'amdates': ['October 16, 2010'], 'eurodates': ['7 May 2005']}

In [97]:
import datetime
eurogroups = re.compile(r"(?P<day>\d{1,2})\w{0,2},? (?P<month>\w{3})\w* (?P<year>\d{4})", re.I)
amgroups = re.compile(r"(?P<month>\w{3})\w* (?P<day>\d{1,2})\w{0,2},? (?P<year>\d{4})", re.I)
monthhash = {'jan': 1,
            'feb': 2,
            'mar': 3,
            'apr': 4,
            'may': 5,
            'jun': 6,
            'jul': 7,
            'aug': 8,
            'sep': 9,
            'oct': 10,
            'nov': 11,
            'dec': 12}
def ProcessDates(dict_from_gdfa):
    pulldate = datetime.date(2016,7,9)
    recentInterval = datetime.timedelta(days=60)
    dates = []
    recent_date_strings = []
    for amdate in dict_from_gdfa['amdates']:
        m = re.search(amgroups, amdate)
        parsedDate = ParseDateMatch(m)
        if parsedDate:
            dates.append(parsedDate)
        else: 
            print("{}: no date found".format(amdate))
        if parsedDate and pulldate - parsedDate <= recentInterval:
            recent_date_strings.append(amdate)
    for eudate in dict_from_gdfa['eurodates']:
        m = re.search(eurogroups, eudate)
        parsedDate = ParseDateMatch(m)
        if parsedDate:
            dates.append(parsedDate)
        else:
            print("{} no date found".format(eudate))
        if parsedDate and pulldate - parsedDate <= recentInterval:
            recent_date_strings.append(eudate)
    if dates:
        mostrecent = max(dates)
        dateCount = len(dates)
        dates.sort()
        if dateCount == 0:
            medianDate = dates[0]
        elif (dateCount % 2 == 1):
            medianIndex = (dateCount - 1) // 2
            medianDate = dates[medianIndex]
        else:
            medianIndices = (dateCount//2, dateCount//2 - 1)
            medianDate = dates[medianIndices[0]] + (dates[medianIndices[1]] - dates[medianIndices[0]]) / 2
        return {'mostRecentDate': mostrecent,
                'medianDate': medianDate,
                'recentDates': recent_date_strings}
    else:
        return None
        
        
def ParseDateMatch(match_object):
    if match_object:
        try:
            d = datetime.date(year=int(match_object.group('year')),
                          month=monthhash[match_object.group('month').lower()],
                          day=int(match_object.group('day')))
            return d
        except ValueError as v:
            print("{}-{}-{}: {}".format(match_object.group('year'),monthhash[match_object.group('month').lower()],
                                      match_object.group('day'), v))
            return None
    else:
        return None
    

In [65]:
ProcessDates(test)

{'medianDate': datetime.date(2008, 1, 26),
 'mostRecentDate': datetime.date(2010, 10, 16),
 'recentDates': []}

In [79]:
list(foundDates.keys())[20]

'910990'

In [103]:
processedDates = dict()
for page_id in foundDates.keys():
    dates_in_id = ProcessDates(foundDates[page_id])
    if dates_in_id:
        processedDates[page_id] = dates_in_id

2006-2-51: day is out of range for month
5135 February 2006 no date found
2005-6-31: day is out of range for month
June 31, 2005: no date found
1821-6-31: day is out of range for month
31 June 1821 no date found
1900-11-0: day is out of range for month
November 0f 1900: no date found
2003-2-29: day is out of range for month
29 February 2003 no date found
1900-1-0: day is out of range for month
January 0, 1900: no date found
1900-2-29: day is out of range for month
February 29, 1900: no date found
1877-3-91: day is out of range for month
March 91, 1877: no date found
1960-5-44: day is out of range for month
4449 May 1960 no date found
1960-9-45: day is out of range for month
4502 Sep 1960 no date found
1961-3-45: day is out of range for month
4597 Mar 1961 no date found
1961-4-46: day is out of range for month
4625 Apr 1961 no date found
1961-8-32: day is out of range for month
323 Aug 1961 no date found
1964-11-51: day is out of range for month
514 Nov 1964 no date found
1962-1-36: day

In [104]:
processedDates.keys()

dict_keys(['4968300', '2970896', '1487450', '3712167', '37421706', '27424121', '5354369', '10467346', '93791', '23489383', '24675992', '18745159', '21167095', '27322852', '346564', '5838620', '48998419', '34958254', '38108033', '41271789', '1707792', '8242057', '13930', '2497678', '29399822', '5876413', '43342282', '31580211', '28731902', '3635831', '43825944', '519437', '7954867', '48935085', '6184863', '28139159', '3630035', '48082088', '33948915', '17901688', '9432075', '33059725', '12964276', '47532106', '15479207', '575924', '18255862', '32307637', '38038901', '41042996', '446099', '38613522', '2345557', '8850976', '24542522', '40787003', '29831', '2703703', '1200251', '638837', '900394', '3572398', '4321352', '221220', '45503803', '12098218', '795617', '41909919', '35495738', '50060897', '8375628', '11103012', '15645057', '43311238', '1291431', '41994809', '34360', '14643713', '25856', '12481251', '45300373', '3042539', '14284113', '44656774', '21192407', '42954395', '48278254', 

In [108]:
processedDates['29831']

{'medianDate': datetime.date(1936, 11, 2),
 'mostRecentDate': datetime.date(2012, 10, 17),
 'recentDates': []}

In [109]:
foundDates['29831']

{'1 April 1908': ' claimed by him, and was patented in Germany on 31 March 1908, patent № 197183, then in Britain, on 1 April 1908, patent № 7219, in France (patent № 390326) and in Russia in 1910 (patent № 17912).',
 '1 January 1954': 'e marketplace. The first national color broadcast (the 1954 Tournament of Roses Parade) occurred on 1 January 1954, but during the following ten years most network broadcasts, and nearly all local programming, cont',
 '1 June 1941': ' its first field test of color television on 20 February 1941. CBS began daily color field tests on 1 June 1941. These color systems were not compatible with existing black-and-white television sets, and, as no ',
 '1 October 2009': 'As of 1 October 2009, the responsibilities held by the BCI are gradually being transferred to the Broadcasting Authority',
 '10 August 1928': "opic without the need of glasses. Stereoscopic 3D television was demonstrated for the first time on 10 August 1928, by John Logie Baird in his company's